# Favorite Song Selection

### Imports

In [ ]:
from bs4 import BeautifulSoup
import requests
import random as r

## Get Album URL from MusicBrainz

In [ ]:
url = input("Enter the url for the album")
data = requests.get(url)

## Get Track Titles


If there are too many tracks (100+) it only loads the first 100

In [ ]:
web_data = BeautifulSoup(data.content,"html.parser")

### Filter Data to get Track Titles

In [ ]:
# This gets the track listing
first_pass = web_data.find_all(class_="title wrap-anywhere")
second_pass = [result.find_all("a")[0] for result in first_pass]
track_list = [result.find_all("bdi")[0].text for result in second_pass]

## Pick Favorite Song

In [ ]:
def favorite_song(tracks):
    available = [song for song in tracks]

    for i in range(len(available)-1):
        choices = r.sample(available,2)

        selection = input(f"{choices[0]} vs {choices[1]} (A vs B)").upper()

        if selection == "A":
            available.remove(choices[1])
        elif selection == "B":
            available.remove(choices[0])
        else:
            raise ValueError(f"Invalid Input ({selection}): Enter either A or B. ")
        
    return available[0]

In [ ]:
def top_n_songs(fav_song,tracks,n):
    song_list = [song for song in tracks]
    favorite_songs = []
    for song in range(n):
        nth_fav = fav_song(song_list)
        song_list.remove(nth_fav)
        favorite_songs.append(nth_fav)
        print(f"#{song+1} song: {nth_fav}") # Print out song choices as they are made
    
    for index,val in enumerate(favorite_songs):
        print(f"{index+1} : {val}") # Print out song ranking

## Choose Top n songs

In [ ]:
top_n_songs(favorite_song,track_list,n=1)